In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
df = pd.read_csv('/Users/tiziana/Desktop/PYTHON/TESI MAGISTRALE/DATI/INFLATION RATE/Consumer price indices (growth rate 1yr).csv')
df.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,METHODOLOGY,Methodology,...,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,BASE_PER,Base period,DURABILITY,Durability,DECIMALS,Decimals
0,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_ALL(1.0),"Consumer price indices (CPIs, HICPs), COICOP 1999",I,AUT,Austria,M,Monthly,N,National,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
1,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_ALL(1.0),"Consumer price indices (CPIs, HICPs), COICOP 1999",I,AUT,Austria,M,Monthly,N,National,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
2,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_ALL(1.0),"Consumer price indices (CPIs, HICPs), COICOP 1999",I,AUT,Austria,M,Monthly,N,National,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
3,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_ALL(1.0),"Consumer price indices (CPIs, HICPs), COICOP 1999",I,AUT,Austria,M,Monthly,N,National,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
4,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_ALL(1.0),"Consumer price indices (CPIs, HICPs), COICOP 1999",I,AUT,Austria,M,Monthly,N,National,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two


In [37]:
df = df[['Reference area','TIME_PERIOD','OBS_VALUE']]

In [38]:
df

,Reference area,TIME_PERIOD,OBS_VALUE
0,Austria,2020-06,1.052830
1,Austria,2020-05,0.640786
2,Austria,2000-06,2.732240
3,Austria,2000-05,1.745296
4,Austria,2000-04,1.857923
...,...,...,...
4117,Greece,2020-07,-1.808604
4118,Finland,2020-07,0.591773
4119,Portugal,2021-02,0.481608
4120,Netherlands,2020-08,0.707833


In [39]:
df.set_index('Reference area',inplace=True)

### Riordino la colonna dei mesi

In [40]:
country = df.groupby('Reference area').sum().index
country

/var/folders/dh/zp5lxqxj269938btf9rcxlh40000gn/T/ipykernel_1425/1398689340.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country = df.groupby('Reference area').sum().index


Index(['Austria', 'Belgium', 'Finland', 'France', 'Germany', 'Greece',
       'Ireland', 'Italy', 'Japan', 'Netherlands', 'Portugal', 'Spain',
       'Sweden', 'United Kingdom', 'United States'],
      dtype='object', name='Reference area')

In [41]:
df1 = {}
for nazione in country:
    dati_nazionali = df.loc[nazione]
    dati_nazionali.set_index('TIME_PERIOD',inplace=True)
    dati_nazionali = dati_nazionali.sort_index()
    
    mean = dati_nazionali.mean()
    std = dati_nazionali.std()

    dati_nazionali = (dati_nazionali - mean)/std

    dati_nazionali['Country'] = str(nazione)
    
    df1[nazione] = dati_nazionali

#### LA PARTE PIU IMPORTANTE
df1 = pd.concat(df1.values()
                     ,axis=0)

In [42]:
df1

,OBS_VALUE,Country
TIME_PERIOD,,
2000-01,-0.661587,Austria
2000-02,-0.359739,Austria
2000-03,-0.174448,Austria
2000-04,-0.224982,Austria
2000-05,-0.294418,Austria
...,...,...
2022-08,3.208546,United States
2022-09,3.174598,United States
2022-10,2.920779,United States


In [44]:
df1.reset_index(inplace=True)
df1['TIME_PERIOD'] = pd.to_datetime(df1['TIME_PERIOD'])
df1['Trimestre'] = df1['TIME_PERIOD'].dt.to_period('Q')




-0.3985914761797981

In [61]:
df_trimestrale = df1.groupby(['Country', 'Trimestre']).agg({'OBS_VALUE': 'mean'}).reset_index()
df_trimestrale

,Country,Trimestre,OBS_VALUE
0,Austria,2000Q1,-0.398591
1,Austria,2000Q2,-0.068451
2,Austria,2000Q3,0.391685
3,Austria,2000Q4,0.373868
4,Austria,2001Q1,0.344333
...,...,...,...
1369,United States,2021Q4,2.333113
1370,United States,2022Q1,3.042635
1371,United States,2022Q2,3.414725
1372,United States,2022Q3,3.245839


In [64]:
df_trimestrale.to_csv('Inflation rate.csv')